In [1]:
import json
import openai
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

load_dotenv()

False

In [2]:


documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [3]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

ObjectApiResponse({'name': '3c78778d3d0a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sMblFZ7TTVSam9nuOGxCfQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [28]:
index_settings = {"settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "faq_search"

es_client.indices.create(index = index_name, body = index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_search'})

In [29]:
for doc in tqdm(documents):
    
    es_client.index(index = index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [30]:
def search(query, coursefilter = "data-engineering-zoomcamp"):

    search_settings =  {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": coursefilter
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body = search_settings)

    result_docs = []
   
    #print(response)
    for hit in response['hits']['hits']:

        result_docs.append(hit['_source'])
    
    return result_docs

In [31]:
def build_prompt(query, related_docs):
    

    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    print("context:",context)
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [32]:
def chat(prompt, model = "phi3"):
   
    client = openai.OpenAI(
        base_url = "http://localhost:11434/v1/",
        
        api_key = "ollama"
    )
    
    response = client.chat.completions.create(
        model = model,
        messages = [{"role":"system", "content":"you are a faq assistant"},
                    {"role": "user", "content": prompt}]
    )
    
    print(response)
    
    return response.choices[0].message.content

In [33]:
def rag(query, model = "phi3"):

    results = search(query = query)
    prompt = build_prompt(query = query, related_docs = results)

    answer = chat(prompt, model = model)

    return answer

In [34]:
query = "I just found out the course has started can i still join it ?"
answer = rag( query = query)

context: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 

section: General course-related questions
question: Course - What can I do before the course starts?
answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anacon

In [35]:
answer

"As per the provided context regarding your course start question, it is generally permissible for students to join a class even after its official starting date as long as they meet any set prerequisites and requirements (dependencies). Nevertheless, please be mindful of final project deadlines if you choose not to register beforehand. \n\nPlease note that your query about joining the course after it starts is missing specific details in regards to context information or instruction relating directly from this document alone; thus additional clarification might help provide a precise answer for better assistance on whether and how late one can officially join an online class within this platform's teaching approach. Based on common academic norms, exceptions are typically rare. \n\u200dstill, the referenced context emphasizes that even if you start late (like after registering), as long as your questions align with any existing FAQ or guidance in relation to prerequisites and course r